In [ ]:
# -----------------------------
# Step 1: Install dependencies
# -----------------------------
!pip install rasterio[all] gdal matplotlib numpy

# -----------------------------
# Step 2: Import libraries
# -----------------------------
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import subprocess

# -----------------------------
# Step 3: Helper - convert JP2 to TIF if needed
# -----------------------------
def convert_to_tif(input_file):
    """Convert JP2 to TIF if rasterio cannot open JP2."""
    output_file = input_file.replace(".jp2", ".tif")
    if not os.path.exists(output_file):
        print(f"Converting {input_file} → {output_file} ...")
        cmd = ["gdal_translate", input_file, output_file]
        subprocess.run(cmd, check=True)
    return output_file

# -----------------------------
# Step 4: Try opening JP2, else fallback to TIF
# -----------------------------
def open_band(file):
    try:
        return rasterio.open(file)
    except:
        tif_file = convert_to_tif(file)
        return rasterio.open(tif_file)

# Replace with your Sentinel band filenames
red_band   = "B04.jp2"   # Red
nir_band   = "B08.jp2"   # Near Infrared
green_band = "B03.jp2"   # Green

red   = open_band(red_band)
nir   = open_band(nir_band)
green = open_band(green_band)

# -----------------------------
# Step 5: Calculate indices
# -----------------------------
red_data   = red.read(1).astype(float)
nir_data   = nir.read(1).astype(float)
green_data = green.read(1).astype(float)

# NDVI = (NIR - RED) / (NIR + RED)
ndvi = (nir_data - red_data) / (nir_data + red_data + 1e-6)

# NDWI = (GREEN - NIR) / (GREEN + NIR)
ndwi = (green_data - nir_data) / (green_data + nir_data + 1e-6)

# -----------------------------
# Step 6: Show plots
# -----------------------------
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

show((red, 1), ax=axes[0], title="Red Band")
im1 = axes[1].imshow(ndvi, cmap='RdYlGn')
axes[1].set_title("NDVI")
plt.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

im2 = axes[2].imshow(ndwi, cmap='Blues')
axes[2].set_title("NDWI")
plt.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

plt.show()

# -----------------------------
# Step 7: Print statistics
# -----------------------------
print("📊 NDVI Stats → min:", np.nanmin(ndvi), "max:", np.nanmax(ndvi), "mean:", np.nanmean(ndvi))
print("📊 NDWI Stats → min:", np.nanmin(ndwi), "max:", np.nanmax(ndwi), "mean:", np.nanmean(ndwi))
